# SQL Aggregations with postgreSQL and sqlalchemy

In [1]:
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.sql import select
import pandas as pd

In [2]:
con = create_engine('postgresql://postgres@localhost:5432/postgres')

In [3]:
con.table_names()

['accounts', 'orders', 'region', 'sales_reps', 'web_events']

In [4]:
accounts = pd.read_csv('accounts.csv', sep=';', index_col='id')
accounts.to_sql('accounts', con=con, if_exists='replace')
orders = pd.read_csv('orders.csv', sep=';', index_col='id')
orders.to_sql('orders', con=con, if_exists='replace')
region = pd.read_csv('region.csv', sep=';', index_col='id')
region.to_sql('region', con=con, if_exists='replace')
sales_reps = pd.read_csv('sales_reps.csv', sep=';', index_col='id')
sales_reps.to_sql('sales_reps', con=con, if_exists='replace')
web_events = pd.read_csv('web_events.csv', sep=';', index_col='id')
web_events.to_sql('web_events', con=con, if_exists='replace')

### Which account (by name) placed the earliest order? 

In [5]:
sql_result = pd.read_sql("\
SELECT a.name, o.occurred_at \
FROM accounts a \
JOIN orders o \
ON a.id = o.account_id \
ORDER BY occurred_at \
LIMIT 1; \
", con)
sql_result.head()

,name,occurred_at
0,DISH Network,2013-12-04T04:22:44.000Z


### Find the total sales in usd for each account. 

In [6]:
sql_result = pd.read_sql("\
SELECT a.name, SUM(total_amt_usd) total_sales \
FROM orders o \
JOIN accounts a \
ON a.id = o.account_id \
GROUP BY a.name; \
", con)
sql_result.head()

,name,total_sales
0,Comcast,12868.38
1,Microsoft,15454.01
2,Monsanto,130964.11
3,Dean Foods,26514.93
4,KKR,217473.85


### Via what channel did the most recent (latest) web_event occur, which account was associated with this web_event?

In [7]:
sql_result = pd.read_sql("\
SELECT w.occurred_at, w.channel, a.name \
FROM web_events w \
JOIN accounts a \
ON w.account_id = a.id ORDER BY w.occurred_at DESC \
LIMIT 1; \
",con)
sql_result.head()

,occurred_at,channel,name
0,2017-01-01T23:51:09.000Z,organic,Molina Healthcare


### Find the total number of times each type of channel from the web_events was used. 

In [8]:
sql_result = pd.read_sql("\
SELECT w.channel, COUNT(*) \
FROM web_events w \
GROUP BY w.channel; \
",con)
sql_result.head()

,channel,count
0,twitter,474
1,adwords,906
2,organic,952
3,banner,476
4,facebook,967


### Who was the primary contact associated with the earliest web_event?

In [9]:
sql_result = pd.read_sql("\
SELECT a.primary_poc \
FROM web_events w \
JOIN accounts a \
ON a.id = w.account_id \
ORDER BY w.occurred_at \
LIMIT 1; \
",con)
sql_result.head()

,primary_poc
0,Leana Hawker


### Find the number of sales reps in each region. 

In [10]:
sql_result = pd.read_sql("\
SELECT r.name, COUNT(*) num_reps \
FROM region r \
JOIN sales_reps s \
ON r.id = s.region_id \
GROUP BY r.name \
ORDER BY num_reps; \
",con)
sql_result.head()

,name,num_reps
0,Midwest,9
1,Southeast,10
2,West,10
3,Northeast,21


### For each account, determine the average amount of each type of paper they purchased across their orders.

In [11]:
sql_result = pd.read_sql("\
SELECT a.name, AVG(o.standard_qty) avg_stand, AVG(o.gloss_qty) avg_gloss, AVG(o.poster_qty) avg_post \
FROM accounts a \
JOIN orders o \
ON a.id = o.account_id \
GROUP BY a.name; \
",con)
sql_result.head()

,name,avg_stand,avg_gloss,avg_post
0,Comcast,294.000000,18.285714,28.857143
1,Microsoft,88.461538,32.230769,62.307692
2,Monsanto,353.614035,42.719298,26.245614
3,Dean Foods,206.857143,214.428571,141.571429
4,KKR,194.490909,229.927273,155.345455


### Determine the number of times a particular channel was used in the web_events table for each sales rep

In [12]:
sql_result = pd.read_sql("\
SELECT s.name, w.channel, COUNT(*) num_events \
FROM accounts a \
JOIN web_events w \
ON a.id = w.account_id \
JOIN sales_reps s \
ON s.id = a.sales_rep_id \
GROUP BY s.name, w.channel \
ORDER BY num_events DESC; \
",con)
sql_result.head()

,name,channel,num_events
0,Earlie Schleusner,direct,234
1,Vernita Plump,direct,232
2,Moon Torian,direct,194
3,Georgianna Chisholm,direct,188
4,Tia Amato,direct,185


### How many of the sales reps have more than 5 accounts that they manage?

In [13]:
sql_result = pd.read_sql("\
SELECT s.id, s.name, COUNT(*) num_accounts \
FROM accounts a \
JOIN sales_reps s \
ON s.id = a.sales_rep_id \
GROUP BY s.id, s.name \
HAVING COUNT(*) > 5 \
ORDER BY num_accounts; \
",con)
sql_result.head()

,id,name,num_accounts
0,321500,Samuel Racine,6
1,321510,Eugena Esser,6
2,321580,Sibyl Lauria,6
3,321590,Necole Victory,6
4,321560,Elba Felder,6


### How many accounts spent more than 30,000 usd total across all orders?

In [14]:
sql_result = pd.read_sql("\
SELECT a.id, a.name, SUM(o.total_amt_usd) total_spent \
FROM accounts a \
JOIN orders o \
ON a.id = o.account_id \
GROUP BY a.id, a.name \
HAVING SUM(o.total_amt_usd) > 30000 \
ORDER BY total_spent; \
",con)
sql_result.head()

,id,name,total_spent
0,1661,American Airlines Group,30083.18
1,1431,PepsiCo,30095.72
2,3661,Group 1 Automotive,30708.92
3,1141,Costco,30741.01
4,1761,Oracle,31231.56


### Which accounts used facebook as a channel to contact customers more than 6 times?

In [15]:
sql_result = pd.read_sql("\
SELECT a.id, a.name, w.channel, COUNT(*) use_of_channel \
FROM accounts a \
JOIN web_events w \
ON a.id = w.account_id \
GROUP BY a.id, a.name, w.channel \
HAVING COUNT(*) > 6 AND w.channel = 'facebook' \
ORDER BY use_of_channel; \
",con)
sql_result.head()

,id,name,channel,use_of_channel
0,1271,Home Depot,facebook,7
1,4291,Avis Budget Group,facebook,7
2,1701,Best Buy,facebook,7
3,3261,Farmers Insurance Exchange,facebook,7
4,3411,Leucadia National,facebook,7


### Find the sales in terms of total dollars for all orders in each year, ordered from greatest to least. Do you notice any trends in the yearly sales totals?

In [18]:
# sql_result = pd.read_sql("\
# SELECT DATE_PART('year', occurred_at) ord_year,  SUM(total_amt_usd) total_spent \
#  FROM orders \
#  GROUP BY 1 \
#  ORDER BY 2 DESC;\
# ",con)
# sql_result.head()

### Write a query to display for each order, the account ID, total amount of the order, and the level of the order - ‘Large’ or ’Small’ - depending on if the order is 500 or more, or less than 500.

In [19]:
sql_result = pd.read_sql("\
SELECT account_id, total, \
CASE WHEN total > 500 THEN 'Large' \
ELSE 'Small' END AS order_level \
FROM orders; \
",con)
sql_result.head()

,account_id,total,order_level
0,1001,169,Small
1,1001,288,Small
2,1001,132,Small
3,1001,176,Small
4,1001,165,Small
